In [3]:
import torch

In [4]:
!pip install torch

In [5]:
import torch
print(torch.__version__)

2.9.1


# Summary of Lab:
# 1. Input Tutoring Session

# 2. Text Preprocessing with Torch with TorchText
# The text is tokeniszed
# Pretrained Glove word embeddings loading

# 3. Using GPT - 2 for text Generation
# A prompt is created that includes the tutoring session and a request
# The model generates a response based on the prompt using beam search
# The result is printed

In [8]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 423.3 kB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchtext]


In [11]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 2.0 MB/s  0:00:06 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 2.4 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.0 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [transformers] [transformers]


In [12]:
# import libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [19]:
# TorchText Preprocessing

import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe

# Define input text
session_text = "This is a sample sentence for embedding."

# Tokenize text
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(session_text)

# Load pretrained GloVe embeddings (100-dim)
glove = GloVe(name="6B", dim=100)

# Filter tokens that exist in GloVe vocab
token_vectors = [glove[token] for token in tokens if token in glove.stoi]

# Mean pooling
if token_vectors:
    session_embedding = torch.stack(token_vectors).mean(dim=0)
else:
    session_embedding = torch.zeros(100)

print(f"Session embedding vector shape: {session_embedding.shape}")

.vector_cache/glove.6B.zip: 862MB [05:39, 2.54MB/s]                                
100%|█████████▉| 399999/400000 [00:05<00:00, 77387.84it/s]


Session embedding vector shape: torch.Size([100])


In [20]:
# --------------------------
# 3. Use an LLM for Generation
# --------------------------
llm_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
llm_model = GPT2LMHeadModel.from_pretrained("gpt2")
llm_model.eval()

# Add prompt for the model
prompt = (
    "Tutoring session summary:\n"
    f"{session_text}\n"
    "Please suggest exercises to improve understanding of identifying the coefficients for quadratic equations.\n" 
    "\n"
    "Summary and question:"
)

# Encode the prompt and generate
inputs = llm_tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

generated = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n--- GPT-2 Output ---\n")
print(generated)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- GPT-2 Output ---

Tutoring session summary:
This is a sample sentence for embedding.
Please suggest exercises to improve understanding of identifying the coefficients for quadratic equations.

Summary and question: What is the relationship between the coefficient of variation and the rate of change in the time series of a given variable? What are the parameters of the model? How do they relate to each other? Is there a correlation between these parameters? Are there any other parameters that can be used to predict the response of an individual variable to changes in time? If so, how can we use them to help us better understand the nature of our models? The answer to these questions is that there are many variables that
